<a href="https://colab.research.google.com/github/Mohammed-Mohra/NLp-Word-Based-/blob/main/NL_word_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import torch
import torch.nn as nn
import unicodedata
import string
import numpy
import re
import pandas as pd
import numpy as np
import keras
import string
import nltk
import gensim, logging
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences

string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '') #why replace the dot with empty shit. Because we want to get senteneces make senseb

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Loads the data and preprocesses data and stores corpus in raw_text
raw_text = open('/content/gdrive/MyDrive/corpus (3).txt', encoding = 'utf8').read()
import nltk
nltk.download('punkt')

nltk.download('stopwords')

# load stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))

file_nl_remo = ""
for line in raw_text:
  new_line = line.replace("\n", " ")           
#removes newlines
  file_nl_remo += new_line

#new_file = new_file.lower()   
 
file_p = re.sub("[^A-Za-z'.]+", ' ',file_nl_remo).lower()   

#removes all special characters
sents = nltk.sent_tokenize(file_p)
print("The number of sentences is", len(sents)) 
#prints the number of sentences

string.punctuation = string.punctuation + '.' # why did we add the dot now 
file_p = re.sub("[^A-Za-z']+", ' ',file_p).lower()   


words = nltk.word_tokenize(file_p)
words = [word for word in words if word not in stop_words]

print("The number of tokens is", len(words)) 
#prints the number of tokens

average_tokens = round(len(words)/len(sents))
print("The average number of tokens per sentence is", average_tokens) 
#prints the average number of tokens per sentence

unique_tokens = set(words)
print("The number of unique tokens are", len(unique_tokens)) 
#prints the number of unique tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The number of sentences is 982
The number of tokens is 12239
The average number of tokens per sentence is 12
The number of unique tokens are 2421


In [ ]:
# Hyperparameters of the model

vocab_size = 2450  #chosen based on statistics of the model

oov_tok = '<OOV>'

embedding_dim = 100

padding_type='post'

trunc_type='post'
# tokenizes sentences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts([file_p])

word_index = tokenizer.word_index
seq_length = 50

tokens = tokenizer.texts_to_sequences([file_p])[0]

In [ ]:
dataX = []
dataY = []

seq_length = 50

for i in range(0, len(tokens) - seq_length-1 ):
    seq_in = tokens[i:i + seq_length] # The 50 word sequence
    seq_out = tokens[i + seq_length] # The word that we want to predict 

    if seq_out==1: #Skip samples where target word is OOV
        continue
    
    dataX.append(seq_in)
    dataY.append(seq_out)

X = numpy.array(dataX)
y = numpy.array(dataY)

X_tensor = torch.tensor(X)
Y_tensor = torch.Tensor(y)
#print ("Total training data size is -", N)
#X = word_vectors
#X = numpy.array(dataX)

# one hot encodes the output variable
#y = numpy.array(dataY)
#y = np_utils.to_categorical(dataY)

In [ ]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 100 # what is the hidden dimension 
LAY_NUM = 2
BATCH_SIZE = 1
class LSTM_model(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size ):
        super(LSTM_model, self).__init__()
        self.hidden_dim = HIDDEN_DIM
        self.seq_len = 50
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim ,num_layers = LAY_NUM, bidirectional = True)

        # The linear layer that maps from hidden state space to tag space
        self.final_output = nn.Linear(2*HIDDEN_DIM, vocab_size)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, word, h_c):
        h_o = h_c[0]  
        c_o = h_c[1]
        embeds = self.word_embeddings(word)
        lstm_out, (h_next,c_next) = self.lstm(embeds.view( self.seq_len, BATCH_SIZE,embeds.shape[-1]), (h_o,c_o))
        lstm_out = lstm_out[49,:]
        lstm_out = self.final_output(lstm_out )

       # lstm_out = torch.relu(lstm_out)
        #print(lstm_out[:20])
        lstm_out = self.softmax(lstm_out)
        #print(lstm_out.shape)

        return lstm_out, (h_next,c_next) # check if lstm_out is same as h_next they should be identical 
    
    def initHidden(self):
        return torch.zeros(2*LAY_NUM,BATCH_SIZE, self.hidden_dim)

In [ ]:
criterion = nn.NLLLoss()


In [ ]:
learning_rate = 0.001 # If you set this too high, it might explode. If too low, it might not learn
lstm_test = 0
y_test = 0

def train(category_tensor, line_tensor):
    h_next = LSTM_model1.initHidden()
    c_next = h_next
    
    LSTM_model1.zero_grad()

    #for i in range(seq_length):
    lstm_out, (h_next,c_next) = LSTM_model1(line_tensor,( h_next, c_next)) # Only the final output is the important one we can ignore all others     
    lstm_test = lstm_out
   # out_index = lstm_out.argmax().unsqueeze(0).long()
    category_tensor = category_tensor.long()
    y_test = category_tensor
   
#.view(BATCH_SIZE,len(lstm_out))
    loss = criterion(lstm_out, category_tensor)
    loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate
    # This is a mystery what is happening cause updating the parameters seems to be not automatic
    optimizer = optim.Adam(LSTM_model1.parameters(), lr= learning_rate)
    optimizer.step()
    
   
    return lstm_out, loss.item()

In [ ]:
Y_tensor.shape

torch.Size([27164])

In [ ]:
X_tensor.shape

torch.Size([27164, 50])

In [ ]:
LSTM_model1 = LSTM_model( EMBEDDING_DIM, HIDDEN_DIM, vocab_size)


In [ ]:
EPOCH_NUM = 50
for j in range(EPOCH_NUM):
  for i in range(212):
      _ , loss = train(Y_tensor[i*BATCH_SIZE:i*BATCH_SIZE + BATCH_SIZE], X_tensor[i*BATCH_SIZE:i*BATCH_SIZE + BATCH_SIZE])
      if(i %10 ==0):
        print(loss)
     
  print('------------------------------------------------------------------------------------------------------------')
  print(f'This is Epoch number {j}')
  print('------------------------------------------------------------------------------------------------------------')



<ipython-input-30-ee168907982b>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  lstm_out = self.softmax(lstm_out)


4.397036075592041


In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
reverse_word_map

In [ ]:
#Creates word to idx map using tokenizer.word_index
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Returns the next n words greedily
def next_tokens(input_str, n):
    Y_index = 0
    h_next = LSTM_model1.initHidden()
    c_next = h_next
    
    print ("Seed -",  input_str, sep = '\n\n')
    final_string = ''
    real_string = ''
    for i in range(n):
        #string.punctuation = string.punctuation.replace('.', '')
        #file_q = "".join([char for char in input_str if char not in string.punctuation])   #removes even periods.
        token = tokenizer.texts_to_sequences([input_str])[0]
        X_tensor = torch.tensor(token)
        X_tensor = X_tensor[:50]
       # print(X_tensor.shape)
        #token = numpy.array(token ).reshape((1,-1))
       # print(token.shape)

        with torch.no_grad():

          prediction,_ = LSTM_model1(X_tensor, ( h_next, c_next))
          prediction = prediction.argmax(dim = 1)
         # print(prediction)
         # print(prediction.shape)
          final_string = final_string + reverse_word_map[int(prediction)] + ' ' 
          input_str = input_str + ' ' + reverse_word_map[int(prediction)]
          input_str = ' '.join(input_str.split(' ')[1:])

    
    return final_string, real_string

In [ ]:
a = torch.tensor([4])
int(a)
#reverse_word_map[a[0]]

4

In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."  
output = next_tokens(input_str, 40)
print("\nGenerated string -\n\n", output)
#print("\Real string -\n\n", real_str)
#speficies an unseen input string


Seed -

The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.

Generated string -

 ('to be be be up on her the of and and to to this she alice and with alice out was was the the said there was she she she she very it it it and and the the in ', '')


<ipython-input-30-ee168907982b>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  lstm_out = self.softmax(lstm_out)
